# Import Library

In [1]:
import pandas as pd
import sqlite3
import requests
import io

conn = sqlite3.connect("./data/chinook.db")

### Get All Country

In [2]:
country = pd.read_sql_query(
    """
    SELECT customers.Country FROM customers
    """, conn)
country["Country"] = pd.DataFrame(country["Country"].unique())
country.dropna()

,Country
0,Brazil
1,Germany
2,Canada
3,Norway
4,Czech Republic
5,Austria
6,Belgium
7,Denmark
8,USA
9,Portugal


## Get all albums

In [4]:
albums = pd.read_sql_query(
        """
            SELECT 
            albums.AlbumId, albums.Title as Album, artists.Name as Artist, tracks.Composer, genres.Name as Genre, tracks.UnitPrice
            FROM tracks
            LEFT JOIN albums ON albums.AlbumId = tracks.AlbumId
            LEFT JOIN artists ON artists.ArtistId = albums.AlbumId
            LEFT JOIN genres ON genres.GenreId = tracks.GenreId
        """
        , conn, index_col="AlbumId")
albums.drop_duplicates(subset=["Album"], keep="first")

,Album,Artist,Composer,Genre,UnitPrice
AlbumId,,,,,
1,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",Rock,0.99
2,Balls to the Wall,Accept,None,Rock,0.99
3,Restless and Wild,Aerosmith,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",Rock,0.99
4,Let There Be Rock,Alanis Morissette,AC/DC,Rock,0.99
5,Big Ones,Alice In Chains,"Steven Tyler, Joe Perry, Jack Blades, Tommy Shaw",Rock,0.99
...,...,...,...,...,...
343,Respighi:Pines of Rome,None,None,Classical,0.99
344,Schubert: The Late String Quartets & String Qu...,None,Franz Schubert,Classical,0.99
345,Monteverdi: L'Orfeo,None,Claudio Monteverdi,Classical,0.99


## Fetch Song Tracks

- [X] Take data from joining minimum of 4 table

In [5]:
tracks = pd.read_sql_query(
 '''
 SELECT 
 tracks.TrackId, tracks.Name as Song, artists.Name as Artist, albums.Title as Album, tracks.Composer, genres.Name as Genre, tracks.UnitPrice
 FROM tracks
 LEFT JOIN albums ON albums.AlbumId = tracks.AlbumId
 LEFT JOIN artists ON artists.ArtistId = albums.AlbumId
 LEFT JOIN genres ON genres.GenreId = tracks.GenreId
 ''', conn)

# tracks.to_csv("./data/tracks.csv", index=False)
tracks.head()

,TrackId,Song,Artist,Album,Composer,Genre,UnitPrice
0,1,For Those About To Rock (We Salute You),AC/DC,For Those About To Rock We Salute You,"Angus Young, Malcolm Young, Brian Johnson",Rock,0.99
1,2,Balls to the Wall,Accept,Balls to the Wall,None,Rock,0.99
2,3,Fast As a Shark,Aerosmith,Restless and Wild,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",Rock,0.99
3,4,Restless and Wild,Aerosmith,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",Rock,0.99
4,5,Princess of the Dawn,Aerosmith,Restless and Wild,Deaffy & R.A. Smith-Diesel,Rock,0.99


In [7]:
top5 = customers.groupby('Country').Total.sum().sort_values(ascending=False).head().index.to_list()
top5_data = customers[customers['Country'].isin(top5)].copy()

dayorder = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


In [8]:
top5_data['InvoiceDOW'] = top5_data['InvoiceDate'].dt.day_name()
top5_data['InvoiceDOW'] = pd.Categorical(top5_data['InvoiceDOW'],
                                         categories=dayorder,
                                         ordered=True)
top5_data

,FullName,Country,InvoiceDate,Total,InvoiceDOW
0,Luís Gonçalves,Brazil,2010-03-11,3.98,Thursday
1,Luís Gonçalves,Brazil,2010-06-13,3.96,Sunday
2,Luís Gonçalves,Brazil,2010-09-15,5.94,Wednesday
3,Luís Gonçalves,Brazil,2011-05-06,0.99,Friday
4,Luís Gonçalves,Brazil,2012-10-27,1.98,Saturday
...,...,...,...,...,...
296,Isabelle Mercier,France,2010-07-15,5.94,Thursday
297,Isabelle Mercier,France,2011-03-05,0.99,Saturday
298,Isabelle Mercier,France,2012-08-26,1.98,Sunday
299,Isabelle Mercier,France,2012-10-06,16.86,Saturday


In [20]:
albums = pd.read_sql_query(
    """
     SELECT 
     albums.AlbumId, albums.Title as Album, artists.Name as Artist, tracks.Composer, genres.Name as Genre, tracks.UnitPrice
     FROM tracks
     LEFT JOIN albums ON albums.AlbumId = tracks.AlbumId
     LEFT JOIN artists ON artists.ArtistId = albums.AlbumId
     LEFT JOIN genres ON genres.GenreId = tracks.GenreId
    """, conn)

albums[albums["Artist"] != "None"]

,AlbumId,Album,Artist,Composer,Genre,UnitPrice
0,1,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",Rock,0.99
1,2,Balls to the Wall,Accept,None,Rock,0.99
2,3,Restless and Wild,Aerosmith,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",Rock,0.99
3,3,Restless and Wild,Aerosmith,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",Rock,0.99
4,3,Restless and Wild,Aerosmith,Deaffy & R.A. Smith-Diesel,Rock,0.99
...,...,...,...,...,...,...
3498,343,Respighi:Pines of Rome,None,None,Classical,0.99
3499,344,Schubert: The Late String Quartets & String Qu...,None,Franz Schubert,Classical,0.99
3500,345,Monteverdi: L'Orfeo,None,Claudio Monteverdi,Classical,0.99
3501,346,Mozart: Chamber Music,None,Wolfgang Amadeus Mozart,Classical,0.99


## Get top album in country

In [19]:
top_albums = pd.read_sql_query(
        """
            SELECT 
            albums.AlbumId, albums.Title as Album, artists.Name as Artist, tracks.Composer, invoices.BillingCountry as Country,
            genres.Name as Genre, tracks.UnitPrice, invoices.Total
            FROM tracks
            LEFT JOIN albums ON albums.AlbumId = tracks.AlbumId
            LEFT JOIN artists ON artists.ArtistId = albums.AlbumId
            LEFT JOIN genres ON genres.GenreId = tracks.GenreId
            LEFT JOIN invoice_items ON invoice_items.InvoiceLineId = tracks.TrackId
            LEFT JOIN invoices ON invoices.InvoiceId = invoice_items.InvoiceLineId
        """, conn)
top_albums[["Country", "Genre"]] = top_albums[["Country", "Genre"]].astype("category", errors="raise")
top_albums["Country"].str.lower()

top_albums

,AlbumId,Album,Artist,Composer,Country,Genre,UnitPrice,Total
0,1,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",Germany,Rock,0.99,1.98
1,2,Balls to the Wall,Accept,None,Norway,Rock,0.99,3.96
2,3,Restless and Wild,Aerosmith,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",Belgium,Rock,0.99,5.94
3,3,Restless and Wild,Aerosmith,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",Canada,Rock,0.99,8.91
4,3,Restless and Wild,Aerosmith,Deaffy & R.A. Smith-Diesel,USA,Rock,0.99,13.86
...,...,...,...,...,...,...,...,...
3498,343,Respighi:Pines of Rome,None,None,NaN,Classical,0.99,NaN
3499,344,Schubert: The Late String Quartets & String Qu...,None,Franz Schubert,NaN,Classical,0.99,NaN
3500,345,Monteverdi: L'Orfeo,None,Claudio Monteverdi,NaN,Classical,0.99,NaN
3501,346,Mozart: Chamber Music,None,Wolfgang Amadeus Mozart,NaN,Classical,0.99,NaN


In [18]:
top_albums = top_albums.groupby(["Country", "Album"])[["Total"]].agg("count").sort_values("Total", ascending=False).reset_index([0,1])
top_albums[top_albums["Country"] == "Argentina"]

,Country,Album,Total
54,Argentina,Prenda Minha,1
184,Argentina,BackBeat Soundtrack,1
191,Argentina,BBC Sessions [Disc 1] [Live],1
214,Argentina,Black Sabbath Vol. 4 (Remaster),1
220,Argentina,Chill: Brazil (Disc 2),1
...,...,...,...
5768,Argentina,Live [Disc 1],0
5769,Argentina,Live On Two Legs [Live],0
5770,Argentina,Live At Donington 1992 (Disc 2),0
5771,Argentina,Live At Donington 1992 (Disc 1),0


## Get Total Invoices

In [12]:
invoice_total = pd.read_sql_query(
                """
                    SELECT InvoiceId, InvoiceDate, (customers.FirstName||' '||customers.LastName) as CustomerName, Country, City, Total
                    FROM invoices
                    LEFT JOIN customers ON customers.CustomerID = invoices.CustomerID
                """, conn)
invoice_total['InvoiceDate'] = pd.to_datetime(invoice_total['InvoiceDate'])
year_order = ["2009", "2010", "2011", "2012", "2013"]
invoice_total['Year'] = invoice_total['InvoiceDate'].dt.year

In [13]:

invoice_total['Year'] = pd.Categorical(invoice_total['Year'], categories=year_order)
invoice_total['Year'] = invoice_total['InvoiceDate'].dt.year
invoice_total = invoice_total.groupby(by=["InvoiceDate", "CustomerName", "Country", "Year"])["Total"].agg("sum").reset_index()

invoice_total.head()

,InvoiceDate,CustomerName,Country,Year,Total
0,2009-01-01,Leonie Köhler,Germany,2009,1.98
1,2009-01-02,Bjørn Hansen,Norway,2009,3.96
2,2009-01-03,Daan Peeters,Belgium,2009,5.94
3,2009-01-06,Mark Philips,Canada,2009,8.91
4,2009-01-11,John Gordon,USA,2009,13.86


In [14]:
mask_year = invoice_total["Year"] == 2010
inv_year = invoice_total[mask_year].melt(id_vars=["Year", "CustomerName"], value_vars="Total", value_name="Total").drop(columns="variable")
inv_year

,Year,CustomerName,Total
0,2010,Isabelle Mercier,1.98
1,2010,Ladislav Kovács,1.98
2,2010,Lucas Mancini,3.96
3,2010,Joakim Johansson,6.94
4,2010,Luis Rojas,17.91
...,...,...,...
78,2010,Johannes Van der Berg,1.98
79,2010,Emma Jones,3.96
80,2010,Diego Gutiérrez,5.94
81,2010,François Tremblay,8.91


## Testing API

In [21]:
albums_url = 'https://capstone-da-api.herokuapp.com/albums'
r = requests.get(albums_url)
r_pd = pd.DataFrame(r.json())
r_pd.head()

,Album,Artist,Composer,Genre,UnitPrice
1,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",Rock,0.99
2,Balls to the Wall,Accept,None,Rock,0.99
3,Restless and Wild,Aerosmith,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",Rock,0.99
4,Let There Be Rock,Alanis Morissette,AC/DC,Rock,0.99
5,Big Ones,Alice In Chains,"Steven Tyler, Joe Perry, Jack Blades, Tommy Shaw",Rock,0.99


In [23]:
top_album_german = 'https://capstone-da-api.herokuapp.com/albums/top/germany'
r = requests.get(top_album_german)
r_pd = pd.DataFrame(r.json())
r_pd.head()

,Country,Album,Total
11,germany,For Those About To Rock We Salute You,4
34,germany,Minha Historia,2
42,germany,Big Ones,2
43,germany,Acústico MTV [Live],2
56,germany,Na Pista,2
